<center>
    <p style="text-align:center">
        <img alt="phoenix logo" src="https://storage.googleapis.com/arize-phoenix-assets/assets/phoenix-logo-light.svg" width="200"/>
        <br>
        <a href="https://arize.com/docs/phoenix/">Docs</a>
        |
        <a href="https://github.com/Arize-ai/phoenix">GitHub</a>
        |
        <a href="https://arize-ai.slack.com/join/shared_invite/zt-2w57bhem8-hq24MB6u7yE_ZF_ilOYSBw#/shared-invite/email">Community</a>
    </p>
</center>
<h1 align="center">Agno Travel Agent Tracing Project</h1>

We will create a simple travel agent powered by the Agno framework and OpenAI models. We’ll begin by installing the necessary OpenInference packages and setting up tracing with Arize.

Next, we’ll define a set of basic tools that provide destination information, estimate trip budgets, and suggest local activities.

Finally, we’ll build and run our agent, viewing the resulting trace outputs in Phoenix to understand how the agent uses its tools and reasoning.

You will need to install Arize Phoenix in your terminal (`pip install arize-phoenix`) an OpenAI API key, and a free [Tavily](https://auth.tavily.com/) API Key.

Ensure you have `phoenix serve` running in your terminal prior to running the following cells. 

# Set up keys and dependenies

In [ ]:
%pip install -qqqqqq arize-phoenix-otel agno openai openinference-instrumentation-agno openinference-instrumentation-openai httpx

In [ ]:
import os
from getpass import getpass

os.environ["OPENAI_API_KEY"] = globals().get("OPENAI_API_KEY") or getpass("🔑 Enter your OpenAI API Key: ")

os.environ["TAVILY_API_KEY"] = globals().get("TAVILY_API_KEY") or getpass("🔑 Enter your Tavily API Key: ")

# Setup tracing

In [ ]:
from phoenix.otel import register

tracer_provider = register(project_name="agno_travel_agent", auto_instrument=True)

# Define tools

First, we’ll define a few helper functions to support our tools. In particular, we’ll use Tavily Search to help the tools gather general information about each destination.

In [ ]:
# --- Helper functions for tools ---
import httpx

def _search_api(query: str) -> str | None:
    """Try Tavily search first, fall back to None."""
    tavily_key = os.getenv("TAVILY_API_KEY")
    if not tavily_key:
        return None
    try:
        resp = httpx.post(
            "https://api.tavily.com/search",
            json={
                "api_key": tavily_key,
                "query": query,
                "max_results": 3,
                "search_depth": "basic",
                "include_answer": True,
            },
            timeout=8,
        )
        data = resp.json()
        answer = data.get("answer") or ""
        snippets = [r.get("content", "") for r in data.get("results", [])]
        combined = " ".join([answer] + snippets).strip()
        return combined[:400] if combined else None
    except Exception:
        return None


def _compact(text: str, limit: int = 200) -> str:
    """Compact text for cleaner outputs."""
    cleaned = " ".join(text.split())
    return cleaned if len(cleaned) <= limit else cleaned[:limit].rsplit(" ", 1)[0]


Our agent will have access to three tools, which we’ll continue to enhance in upcoming labs:

1. Essential Info – Provides key travel details about the destination, such as weather and general conditions.

2. Budget Basics – Offers insights into travel costs and helps plan budgets based on selected activities.

3. Local Flavor – Recommends unique local experiences and cultural highlights.

In [ ]:
from agno.tools import tool

@tool
def essential_info(destination: str) -> str:
    """Get basic travel info (weather, best time, attractions, etiquette)."""
    q = f"{destination} travel essentials weather best time top attractions etiquette"
    s = _search_api(q)
    if s:
        return f"{destination} essentials: {_compact(s)}"

    return f"{destination} is a popular travel destination. Expect local culture, cuisine, and landmarks worth exploring."

@tool
def budget_basics(destination: str, duration: str) -> str:
    """Summarize travel cost categories."""
    q = f"{destination} travel budget average daily costs {duration}"
    s = _search_api(q)
    if s:
        return f"{destination} budget ({duration}): {_compact(s)}"
    return f"Budget for {duration} in {destination} depends on lodging, meals, transport, and attractions."

@tool
def local_flavor(destination: str, interests: str = "local culture") -> str:
    """Suggest authentic local experiences."""
    q = f"{destination} authentic local experiences {interests}"
    s = _search_api(q)
    if s:
        return f"{destination} {interests}: {_compact(s)}"
    return f"Explore {destination}'s unique {interests} through markets, neighborhoods, and local eateries."


# Define agent

Next, we’ll construct our agent. The Agno framework makes this process straightforward by allowing us to easily define key parameters such as the model, instructions, and tools.

In [ ]:
from agno.agent import Agent
from agno.models.openai import OpenAIChat

trip_agent = Agent(
    name="TripPlanner",
    role="AI Travel Assistant",
    model=OpenAIChat(id="gpt-4.1"),
    instructions=(
        "You are a friendly and knowledgeable travel planner. "
        "Combine multiple tools to create a trip plan including essentials, budget, and local flavor. "
        "Keep the tone natural, clear, and under 1000 words."
    ),
    markdown=True,
    tools=[essential_info, budget_basics, local_flavor],
)

# Run agent

Finally, we are ready to run our agent! Run this cell to see an example in action.

In [ ]:
# --- Example usage ---
destination = "Tokyo"
duration = "5 days"
interests = "food, culture"

query = f"""
Plan a {duration} trip to {destination}.
Focus on {interests}.
Include essential info, budget breakdown, and local experiences.
"""
trip_agent.print_response(
    query,
    stream = True
)